In this notebook, we demonstrate how the combined-RPE model can find a positive correlation between brain activity and model-free and model-based reward prediction errors even when no correlation exists.

First, we import some libraries to help us generate the data and perform the analysis. We also set a seed for the random number generator for reproducibility. 

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

np.random.seed(47824)

# Number of data points
N = 10_000

We generate some random data to be the model-based reward prediction error (RPE). The chosen mean and scale are similar to those of our empirical data.

In [ ]:
mb_rpe = np.random.normal(scale=0.5, size=N)

Then we generate the model-free RPE by adding random numbers to the model-based RPE, so that the two RPEs are positively correlated. The model-free RPE will have a higher variance than the model-based RPE, which is also similar to what we see in our empirical data.

In [ ]:
mf_rpe = mb_rpe + np.random.normal(scale=0.5, size=N)

We calculate below the correlation between model-free and the model-based RPEs to check that they are positively correlated.


Note: the `pearsonr` function returns below both the value of the correlation and its P-value.

In [ ]:
print(stats.pearsonr(mf_rpe, mb_rpe))

(0.6984022255948152, 0.0)


Next we generate the RPEs at feedback.

In [ ]:
fb_rpe = np.random.normal(scale=0.5, size=N)

Now we generate some random brain activity for the second stage and check that it is uncorrelated with both the model-free and the model-based RPEs. If by chance there is a correlation, then we repeat the procedure with a different random vector because this demonstration is based on the assumption that the two signals are not related.

In [ ]:
pval = 0.
while pval < 0.2:
  stage2_act = np.random.normal(loc=1., scale=0.5, size=N)
  mfcorr = stats.pearsonr(mf_rpe, stage2_act)
  mbcorr = stats.pearsonr(mb_rpe, stage2_act)
  pval = min(mfcorr[1], mbcorr[1])
print("Correlation with the MF RPE:", mfcorr)
print("Correlation with the MB RPE:", mbcorr)

Correlation with the MF RPE: (0.0063770424440286865, 0.5237139049107337)
Correlation with the MB RPE: (0.008034532259768433, 0.4217633628773688)


We then generate brain activity for feedback by adding normally distributed random numbers to the feedback RPE so that brain activity and RPEs at feedback are positively correlated.

In [ ]:
fb_brain_act = fb_rpe + np.random.normal(loc=2, scale=0.5, size=N)
print(stats.pearsonr(fb_brain_act, fb_rpe))

(0.701671195233547, 0.0)


Next we interleave brain activity at the second stage with brain activity at feedback, then we do the same with the model-free and model-based RPE regressors. The even elements of each array refer to the second stage and the odd elements to feedback. In the fMRI data, all regressors were also convolved with the same HRF. However, convolution with the HRF does not change the basic phenomenon we are demonstrating here, because it is a linear operation and in reality only the correlations between the regressors are relevant to the final result. Therefore, we omit the HRF convolution for simplicity.

In [ ]:
# Function to interleave two vectors
def interleave(a, b):
  c = np.empty((a.size + b.size,), dtype=a.dtype)
  c[0::2] = a
  c[1::2] = b
  return c

brain_act = interleave(stage2_act, fb_brain_act)
mf_rpe_comb = interleave(mf_rpe, fb_rpe)
mb_rpe_comb = interleave(mb_rpe, fb_rpe)

Then we subtract the model-free RPE from the model-based RPE as was done when the combined-RPE GLM was used in previous papers.

In [ ]:
mb_rpe_comb -= mf_rpe_comb

Notice that the even elements of the model-based and the model-free RPE regressors often become negatively correlated following this subtraction. Whether the obtained correlation is negative, positive, or zero depends on the initial data. It is not necessarily true that the correlation will become negative following this procedure for all pairs of regressors, but the correlation was significantly negative in our empirical data and in over 99% of our stimulations.

In [ ]:
print(stats.pearsonr(mb_rpe_comb[0::2], mf_rpe_comb[0::2]))

(-0.7082992992868536, 0.0)


We then mean-center and orthogonalize the combined model-free and the model-based RPEs, which eliminates the bivariate correlation between the entire vectors.

In [ ]:
mf_rpe_comb -= np.mean(mf_rpe_comb)
mb_rpe_comb -= np.mean(mb_rpe_comb)

# Function to orthogonalize two vectors
def orth(u1, u2):
  return (u1, u2 - np.dot(u1, u2)/np.dot(u1, u1)*u1)

mf_rpe_comb, mb_rpe_comb = orth(mf_rpe_comb, mb_rpe_comb)
print(stats.pearsonr(mb_rpe_comb, mf_rpe_comb))

(1.1102230246251565e-16, 0.9999999999961807)


However, the even elements of the regressors are often still negatively correlated. The orthogonalization procedure always removes the correlation between the entire vectors, but the even elements of the regressors may still be correlated.


In [ ]:
print(stats.pearsonr(mb_rpe_comb[0::2], mf_rpe_comb[0::2]))

(-0.31243989705270436, 2.5227586044511936e-225)


We now create separate intercepts for the second-stage events and the feedback events ("onset" regressors).

In [ ]:
stage2_onset = interleave(np.ones(N), np.zeros(N))
fb_onset = interleave(np.zeros(N), np.ones(N))

We then generate a combined onset regressor and create a predictor data frame for the combined-RPE model, containing the two combined-RPE regressors, the combined onset regressor and the feedback onset regressor. Finally, we fit the model to brain activity using least-squares linear regression.

In [ ]:
X = pd.DataFrame({
    'combined_onset': (stage2_onset + fb_onset),
    'feedback_onset': fb_onset,
    'model_free_rpe': mf_rpe_comb,
    'model_based_rpe': mb_rpe_comb
})
model = sm.OLS(brain_act, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.501
Model:                            OLS   Adj. R-squared:                  0.501
Method:                 Least Squares   F-statistic:                     6705.
Date:                Thu, 03 Feb 2022   Prob (F-statistic):               0.00
Time:                        12:33:33   Log-Likelihood:                -16609.
No. Observations:               20000   AIC:                         3.323e+04
Df Residuals:                   19996   BIC:                         3.326e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
combined_onset      1.0025      0.006    1

As can be seen from the above results, the coefficientes for the combined model-free and model-based regressors are both positive, despite the fact that brain activity at the second stage is uncorrelated with either RPE.

In contrast, when we compute the separated-RPE model we find results that are consistent with the ground truth in our simulated data. Specifically, the coefficients for the model-free and model-based regressors are zero (or at least very small and not significantly different from zero), and only the coefficient for the feedback regressor is significantly positive.

In [ ]:
# Creating the separate regressors
mf_rpe_sep = mf_rpe
mb_rpe_sep = mb_rpe - mf_rpe
# Mean centering
mf_rpe_sep -= np.mean(mf_rpe_sep)
mb_rpe_sep -= np.mean(mb_rpe_sep)
# Orthogonalization
mf_rpe_sep, mb_rpe_sep = orth(mf_rpe_sep, mb_rpe_sep)
# Linear regression
X = pd.DataFrame({
    'stage2_onset': stage2_onset,
    'feedback_onset': fb_onset,
    'model_free_rpe': interleave(mf_rpe_sep, np.zeros(N)),
    'model_based_rpe': interleave(mb_rpe_sep, np.zeros(N)),
    'feedback_rpe': interleave(np.zeros(N), fb_rpe),
})
model = sm.OLS(brain_act, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.598
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                     7440.
Date:                Thu, 03 Feb 2022   Prob (F-statistic):               0.00
Time:                        12:33:37   Log-Likelihood:                -14454.
No. Observations:               20000   AIC:                         2.892e+04
Df Residuals:                   19995   BIC:                         2.896e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
stage2_onset        1.0024      0.005    2